In [143]:
import numpy as np 
import pandas as pd 

train = pd.read_csv("./Datasets/train.csv")
test = pd.read_csv("./Datasets/test.csv")

In [144]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [145]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader 
from torch.utils.data import Dataset

In [146]:
class MyDataset(Dataset):
  def __init__(self,df):

    x=df.iloc[:,1:786].values
    y=df.iloc[:,0].values
    self.x_train=torch.tensor(x,dtype=torch.float32)
    self.y_train=F.one_hot(torch.tensor(y,dtype=torch.float32).type(torch.LongTensor)).type(torch.FloatTensor)

 
  def __len__(self):
    return len(self.y_train)
   
  def __getitem__(self,idx):
    return self.x_train[idx],self.y_train[idx]

In [147]:
data=MyDataset(train)
train_loader=DataLoader(data,batch_size=10)

In [148]:
for i, (data, labels) in enumerate(train_loader):
  print(data.shape, labels.shape)
  print(data,labels)
  break;
 

torch.Size([10, 784]) torch.Size([10, 10])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]) tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]])


In [149]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fco = nn.Linear(50, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fco(x)
        x= self.softmax(x)
        return x 

In [155]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = 784 
num_classes = 10
learning_rate = 0.0001
num_epochs = 100

In [156]:
model = NN(input_size=input_size, num_classes=num_classes)#.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
print(model)

NN(
  (fc1): Linear(in_features=784, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fco): Linear(in_features=50, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)


In [157]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy"
            f" {float(num_correct) / float(num_samples) * 100:.2f}"
        )

    model.train()

In [158]:
for epoch in range(num_epochs):
    print(f"Epoch: {epoch}")
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        #data = data.to(device=device)
        #targets = targets.to(device=device)

        # Get to correct shape, 28x28->784
        # -1 will flatten all outer dimensions into one
        data = data.reshape(data.shape[0], -1) 

        # forward propagation
        scores = model(data)
        loss = criterion(scores, targets)

        # zero previous gradients
        optimizer.zero_grad()
        
        # back-propagation
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

        check_accuracy(train_loader, model)


Epoch: 0
Got 271 / 42000 with accuracy 0.65
Got 358 / 42000 with accuracy 0.85
Got 461 / 42000 with accuracy 1.10
Got 656 / 42000 with accuracy 1.56
Got 816 / 42000 with accuracy 1.94
Got 999 / 42000 with accuracy 2.38
Got 1173 / 42000 with accuracy 2.79
Got 1341 / 42000 with accuracy 3.19
Got 1530 / 42000 with accuracy 3.64
Got 1749 / 42000 with accuracy 4.16
Got 1916 / 42000 with accuracy 4.56
Got 2171 / 42000 with accuracy 5.17
Got 2414 / 42000 with accuracy 5.75
Got 2572 / 42000 with accuracy 6.12
Got 2764 / 42000 with accuracy 6.58
Got 2969 / 42000 with accuracy 7.07
Got 3105 / 42000 with accuracy 7.39
Got 3282 / 42000 with accuracy 7.81
Got 3443 / 42000 with accuracy 8.20
Got 3694 / 42000 with accuracy 8.80
Got 3968 / 42000 with accuracy 9.45
Got 4229 / 42000 with accuracy 10.07
Got 4564 / 42000 with accuracy 10.87
Got 4833 / 42000 with accuracy 11.51
Got 4989 / 42000 with accuracy 11.88
Got 5305 / 42000 with accuracy 12.63
Got 5755 / 42000 with accuracy 13.70
Got 6243 / 42000 wi

KeyboardInterrupt: 

In [ ]:
check_accuracy(train_loader, model)

Got 8532 / 42000 with accuracy 20.31
